In [1]:
#params

#linear_scaling_tau
#linear_scaling_tau_intercept
#switch_time_unconstr
#gamma_mean_unconstr
#beta_mean_unconstr
#alpha_unconstr
#alpha_1_unconstr
#lambda_alpha_unconstr
#scale_unconstr
#theta

#encoder weights
#weights from 3 sep decoders (latent time, transcription state, GP)

#does pytroch lightning automatically know to update weigts of decoders/encoders?
#How is prox weights W trained? Is it is aseparete trainer to the decoder/encoder? Are the losses combined into one?

%load_ext autoreload
%autoreload 2


In [2]:
!pip install --quiet scvi-colab
from scvi_colab import install
install()

/home/chels/anaconda3/envs/thesis/lib/python3.8/site-packages/scvi_colab/_core.py:41: UserWarning: 
                Not currently in Google Colab environment.

                Please run with `run_outside_colab=True` to override.

                Returning with no further action.
                
  warn(


In [5]:
import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv
import torch
from velovi import preprocess_data, VELOVI

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
adata = scv.datasets.pancreas()

In [7]:
scv.pp.filter_and_normalize(adata, min_shared_counts=30, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

Filtered out 21611 genes that are detected 30 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.
computing neighbors
    finished (0:00:18) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:01) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [8]:
adata = preprocess_data(adata)

computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [19]:
VELOVI.setup_anndata(adata, spliced_layer="Ms", unspliced_layer="Mu")

Test implementation of model

In [20]:
vae = VELOVI(adata)

	with soft mask.
Last Decoder layer: softmax


In [21]:
vae

VELOVI Model with the following params: 
n_hidden: 256, n_latent: 10, n_layers: 1, dropout_rate: 0.1
Training status: Not Trained

In [22]:
vae.view_anndata_setup()

Anndata setup with scvi-tools version 0.18.0.

Setup via `VELOVI.setup_anndata` with arguments:

{'spliced_layer': 'Ms', 'unspliced_layer': 'Mu'}

     Summary Statistics     
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Summary Stat Key ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│       n_U        │ 1043  │
│     n_cells      │ 3696  │
│      n_vars      │ 1043  │
└──────────────────┴───────┘

            Data Registry             
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃ scvi-tools Location ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│      U       │ adata.layers['Mu']  │
│      X       │ adata.layers['Ms']  │
└──────────────┴─────────────────────┘

In [12]:
vae.train()


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 1/500:   0%|          | 0/500 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (1043) must match the size of tensor b (0) at non-singleton dimension 1

In [28]:
from velovi import VELOVAE

In [32]:
model = VELOVAE()

TypeError: __init__() missing 1 required positional argument: 'n_input'

In [31]:
tensors=adata.layers["Ms"]

In [32]:
model.forward(tensors)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [15]:
adata

AnnData object with n_obs × n_vars = 3696 × 1043
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts'
    var: 'highly_variable_genes', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes'
    uns: 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca', 'velocity_params', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced', 'Ms', 'Mu', 'velocity'
    obsp: 'distances', 'connectivities'

In [17]:
def one_hot_encoder(idx, n_cls):
    assert torch.max(idx).item() < n_cls
    if idx.dim() == 1:
        idx = idx.unsqueeze(1)
    onehot = torch.zeros(idx.size(0), n_cls)
    onehot = onehot.to(idx.device)
    onehot.scatter_(1, idx.long(), 1)
    return onehot

In [19]:
from typing import Callable, Iterable, Optional

import numpy as np
import torch
import torch.nn.functional as F
from scvi._compat import Literal
from scvi.module.base import BaseModuleClass, LossRecorder, auto_move_data
from scvi.nn import Encoder, FCLayers
from torch import nn as nn
from torch.distributions import Categorical, Dirichlet, MixtureSameFamily, Normal
from torch.distributions import kl_divergence as kl
from scvi.distributions import NegativeBinomial

from velovi import REGISTRY_KEYS

torch.backends.cudnn.benchmark = True



class MaskedLinear(nn.Linear):
    def __init__(self, n_in,  n_out, mask, bias=True):
        # mask should have the same dimensions as the transposed linear weight
        # n_input x n_output_nodes
        if n_in != mask.shape[0] or n_out != mask.shape[1]:
            raise ValueError('Incorrect shape of the mask.')

        super().__init__(n_in, n_out, bias)

        self.register_buffer('mask', mask.t())

        # zero out the weights for group lasso
        # gradient descent won't change these zero weights
        self.weight.data*=self.mask

    def forward(self, input):
        return nn.functional.linear(input, self.weight*self.mask, self.bias)

class MaskedCondLayers(nn.Module):
    def __init__(
        self,
        n_in: int,
        n_out: int,
        n_cond: int,
        bias: bool,
        n_ext: int = 0,
        n_ext_m: int = 0,
        mask: Optional[torch.Tensor] = None,
        ext_mask: Optional[torch.Tensor] = None
    ):
        super().__init__()
        self.n_cond = n_cond
        self.n_ext = n_ext
        self.n_ext_m = n_ext_m

        self.expr_L = nn.Linear(n_in, n_out, bias=bias)

        # if mask is None:
        #     self.expr_L = nn.Linear(n_in, n_out, bias=bias)
        # else:
        #     self.expr_L = MaskedLinear(n_in, n_out, mask, bias=bias)

        # if self.n_cond != 0:
        #     self.cond_L = nn.Linear(self.n_cond, n_out, bias=False)

        # if self.n_ext != 0:
        #     self.ext_L = nn.Linear(self.n_ext, n_out, bias=False)

        # if self.n_ext_m != 0:
        #     if ext_mask is not None:
        #         self.ext_L_m = MaskedLinear(self.n_ext_m, n_out, ext_mask, bias=False)
        #     else:
        #         self.ext_L_m = nn.Linear(self.n_ext_m, n_out, bias=False)

    def forward(self, x: torch.Tensor):
        # if self.n_cond == 0:
        #     expr, cond = x, None
        # else:
        #     expr, cond = torch.split(x, [x.shape[1] - self.n_cond, self.n_cond], dim=1)

        # if self.n_ext == 0:
        #     ext = None
        # else:
        #     expr, ext = torch.split(expr, [expr.shape[1] - self.n_ext, self.n_ext], dim=1)

        # if self.n_ext_m == 0:
        #     ext_m = None
        # else:
        #     expr, ext_m = torch.split(expr, [expr.shape[1] - self.n_ext_m, self.n_ext_m], dim=1)

        expr=x

        out = self.expr_L(expr)
        # if ext is not None:
        #     out = out + self.ext_L(ext)
        # if ext_m is not None:
        #     out = out + self.ext_L_m(ext_m)
        # if cond is not None:
        #     out = out + self.cond_L(cond)
        return out

class DecoderVELOVI(nn.Module):
    """
    Decodes data from latent space of ``n_input`` dimensions ``n_output``dimensions.

    Uses a fully-connected neural network of ``n_hidden`` layers.

    Parameters
    ----------
    n_input
        The dimensionality of the input (latent space)
    n_output
        The dimensionality of the output (data space)
    n_cat_list
        A list containing the number of categories
        for each category of interest. Each category will be
        included using a one-hot encoding
    n_layers
        The number of fully-connected hidden layers
    n_hidden
        The number of nodes per hidden layer
    dropout_rate
        Dropout rate to apply to each of the hidden layers
    inject_covariates
        Whether to inject covariates in each layer, or just the first (default).
    use_batch_norm
        Whether to use batch norm in layers
    use_layer_norm
        Whether to use layer norm in layers
    linear_decoder
        Whether to use linear decoder for time
    """

    def __init__(
        self,
        n_input: int,
        n_output: int,
        n_ext: int = 0,
        n_ext_m: int = 0,
        n_cond: int = 0,
        last_layer: str =None,
        ext_mask: torch.Tensor = None,
        mask: torch.Tensor = None,
        recon_loss: str = 'nb',
        n_cat_list: Iterable[int] = None,
        n_layers: int = 1,
        n_hidden: int = 128,
        inject_covariates: bool = True,
        use_batch_norm: bool = True,
        use_layer_norm: bool = False,
        dropout_rate: float = 0.0,
        linear_decoder: bool = False,
        **kwargs,
    ):
        super().__init__()
        self.n_ouput = n_output
        self.linear_decoder = linear_decoder

        ### GP decoder ###

        if recon_loss == "mse":
            if last_layer == "softmax":
                raise ValueError("Can't specify softmax last layer with mse loss.")
            last_layer = "identity" if last_layer is None else last_layer
        elif recon_loss == "nb":
            last_layer = "softmax" if last_layer is None else last_layer
        else:
            raise ValueError("Unrecognized loss.")

        #print("GP Decoder Architecture:")
        #print("\tMasked linear layer in, ext_m, ext, cond, out: ", in_dim, n_ext_m, n_ext, n_cond, out_dim)
        if mask is not None:
            print('\twith hard mask.')
        else:
            print('\twith soft mask.')

        self.n_ext = n_ext
        self.n_ext_m = n_ext_m

        self.n_cond = 0
        if n_cond is not None:
            self.n_cond = n_cond

        self.L0 = MaskedCondLayers(n_input, n_output, n_cond, bias=False, n_ext=n_ext, n_ext_m=n_ext_m,
                                   mask=mask, ext_mask=ext_mask)

        if last_layer == "softmax":
            self.mean_decoder = nn.Softmax(dim=-1)
        elif last_layer == "softplus":
            self.mean_decoder = nn.Softplus()
        elif last_layer == "exp":
            self.mean_decoder = torch.exp
        elif last_layer == "relu":
            self.mean_decoder = nn.ReLU()
        elif last_layer == "identity":
            self.mean_decoder = lambda a: a
        else:
            raise ValueError("Unrecognized last layer.")

        print("Last Decoder layer:", last_layer)

        self.rho_first_decoder = FCLayers(
            n_in=n_input,
            n_out=n_hidden if not linear_decoder else n_output,
            n_cat_list=n_cat_list,
            n_layers=n_layers if not linear_decoder else 1,
            n_hidden=n_hidden,
            dropout_rate=dropout_rate,
            inject_covariates=inject_covariates,
            use_batch_norm=use_batch_norm,
            use_layer_norm=use_layer_norm if not linear_decoder else False,
            use_activation=not linear_decoder,
            bias=not linear_decoder,
            **kwargs,
        )

        self.pi_first_decoder = FCLayers(
            n_in=n_input,
            n_out=n_hidden,
            n_cat_list=n_cat_list,
            n_layers=n_layers,
            n_hidden=n_hidden,
            dropout_rate=dropout_rate,
            inject_covariates=inject_covariates,
            use_batch_norm=use_batch_norm,
            use_layer_norm=use_layer_norm,
            **kwargs,
        )

        self.px_pi_decoder = nn.Linear(n_hidden, 4 * n_output)
        

        # rho for induction
        self.px_rho_decoder = nn.Sequential(nn.Linear(n_hidden, n_output), nn.Sigmoid())

        # tau for repression
        self.px_tau_decoder = nn.Sequential(nn.Linear(n_hidden, n_output), nn.Sigmoid())

        self.linear_scaling_tau = nn.Parameter(torch.zeros(n_output))
        self.linear_scaling_tau_intercept = nn.Parameter(torch.zeros(n_output))

    def forward(self, z: torch.Tensor, latent_dim: int = None):
        """
        The forward computation for a single sample.

         #. Decodes the data from the latent space using the decoder network
         #. Returns parameters for the ZINB distribution of expression
         #. If ``dispersion != 'gene-cell'`` then value for that param will be ``None``

        Parameters
        ----------
        z :
            tensor with shape ``(n_input,)``
        cat_list
            list of category membership(s) for this sample

        Returns
        -------
        4-tuple of :py:class:`torch.Tensor`
            parameters for the ZINB distribution of expression

        """

        z_in = z
        if latent_dim is not None:
            mask = torch.zeros_like(z)
            mask[..., latent_dim] = 1
            z_in = z * mask
        # The decoder returns values for the parameters of the ZINB distribution
        rho_first = self.rho_first_decoder(z_in)

        dec_latent = self.L0(z)
        recon_x = self.mean_decoder(dec_latent)

        if not self.linear_decoder:
            px_rho = self.px_rho_decoder(rho_first)
            px_tau = self.px_tau_decoder(rho_first)
        else:
            px_rho = nn.Sigmoid()(rho_first)
            px_tau = 1 - nn.Sigmoid()(
                rho_first * self.linear_scaling_tau.exp()
                + self.linear_scaling_tau_intercept
            )

        # cells by genes by 4
        pi_first = self.pi_first_decoder(z)
        px_pi = nn.Softplus()(
            torch.reshape(self.px_pi_decoder(pi_first), (z.shape[0], self.n_ouput, 4))
        )

        return px_pi, px_rho, px_tau, recon_x, dec_latent

In [ ]:
n_input_encoder = adata.n_vars*2
n_latent = 10
z_encoder = Encoder(
            n_input_encoder,
            n_latent,
        )

In [ ]:
class VAE(torch.nn.Module):
    def __init__(
        self,
        n_input: int = adata.n_vars,
        n_input_encoder: int= adata.n_vars*2,
        n_output: int = adata.n_vars,
        n_latent: int = 10
    ):

        super().__init__()
        z_encoder = Encoder(
            n_input_encoder,
            n_latent,
        )

        decoder = DecoderVELOVI(n_latent,n_output)
        
    def forward(self, x: torch.Tensor):
        output = self.neural_net(x)
        if self.transformation:
            output = self.transformation(output)
        return output

In [21]:
decoder = DecoderVELOVI(10,adata.n_vars)

	with soft mask.
Last Decoder layer: softmax


In [26]:
a = torch.randn(256,10)

In [27]:
decoder(a)

(tensor([[[0.5960, 0.5593, 0.7746, 0.5581],
          [0.6247, 0.5923, 0.6138, 0.6152],
          [1.0709, 0.6190, 0.9171, 0.8181],
          ...,
          [0.7585, 0.4583, 0.8887, 0.6110],
          [0.7388, 0.7837, 0.5911, 0.6240],
          [0.5901, 0.8256, 0.6934, 0.5461]],
 
         [[0.5082, 0.4603, 0.8663, 0.6320],
          [0.7174, 0.7410, 0.7441, 0.7580],
          [0.8592, 0.5162, 0.7431, 0.5351],
          ...,
          [0.6980, 0.6865, 0.6055, 0.9528],
          [0.6559, 0.5099, 0.7216, 0.6988],
          [0.5626, 0.5031, 0.7445, 0.8555]],
 
         [[0.4554, 0.4885, 1.0618, 1.0480],
          [0.7135, 0.7935, 0.7928, 0.7879],
          [1.1358, 1.0371, 1.2516, 0.8622],
          ...,
          [0.7583, 0.7441, 0.7194, 0.6469],
          [0.6610, 0.5859, 0.9133, 0.6825],
          [1.0145, 0.9254, 0.7130, 0.5718]],
 
         ...,
 
         [[0.4953, 0.3934, 0.6765, 0.6708],
          [0.7536, 0.6493, 0.8151, 1.2298],
          [0.9456, 1.0350, 0.6159, 0.5112],
      

In [ ]:
decoder_out=decoder(a)

In [30]:
import logging
import warnings
from functools import partial
from typing import Iterable, List, Optional, Sequence, Tuple, Union

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from anndata import AnnData
from joblib import Parallel, delayed
from scipy.stats import ttest_ind
from scvi._compat import Literal
from scvi._utils import _doc_params
from scvi.data import AnnDataManager
from scvi.data.fields import LayerField
from scvi.dataloaders import AnnDataLoader, DataSplitter
from scvi.model._utils import scrna_raw_counts_properties
from scvi.model.base import BaseModelClass, UnsupervisedTrainingMixin, VAEMixin
from scvi.model.base._utils import _de_core
from scvi.train import TrainingPlan, TrainRunner
from scvi.utils._docstrings import doc_differential_expression, setup_anndata_dsp
from sklearn.metrics.pairwise import cosine_similarity

from velovi import REGISTRY_KEYS
from velovi import VELOVAE

logger = logging.getLogger(__name__)


def _softplus_inverse(x: np.ndarray) -> np.ndarray:
    x = torch.from_numpy(x)
    x_inv = torch.where(x > 20, x, x.expm1().log()).numpy()
    return x_inv


class VELOVI(VAEMixin, UnsupervisedTrainingMixin, BaseModelClass):
    """
    Velocity Variational Inference

    Parameters
    ----------
    adata
        AnnData object that has been registered via :func:`~velovi.VELOVI.setup_anndata`.
    n_hidden
        Number of nodes per hidden layer.
    n_latent
        Dimensionality of the latent space.
    n_layers
        Number of hidden layers used for encoder and decoder NNs.
    dropout_rate
        Dropout rate for neural networks.
    gamma_init_data
        Initialize gamma using the data-driven technique.
    linear_decoder
        Use a linear decoder from latent space to time.
    **model_kwargs
        Keyword args for :class:`~velovi.VELOVAE`
    """

    def __init__(
        self,
        adata: AnnData,
        n_hidden: int = 256,
        n_latent: int = 10,
        n_layers: int = 1,
        dropout_rate: float = 0.1,
        gamma_init_data: bool = False,
        linear_decoder: bool = False,
        **model_kwargs,
    ):
        super().__init__(adata)
        self.n_latent = n_latent

        spliced = self.adata_manager.get_from_registry(REGISTRY_KEYS.X_KEY)
        unspliced = self.adata_manager.get_from_registry(REGISTRY_KEYS.U_KEY)

        sorted_unspliced = np.argsort(unspliced, axis=0)
        ind = int(adata.n_obs * 0.99)
        us_upper_ind = sorted_unspliced[ind:, :]

        us_upper = []
        ms_upper = []
        for i in range(len(us_upper_ind)):
            row = us_upper_ind[i]
            us_upper += [unspliced[row, np.arange(adata.n_vars)][np.newaxis, :]]
            ms_upper += [spliced[row, np.arange(adata.n_vars)][np.newaxis, :]]
        us_upper = np.median(np.concatenate(us_upper, axis=0), axis=0)
        ms_upper = np.median(np.concatenate(ms_upper, axis=0), axis=0)

        alpha_unconstr = _softplus_inverse(us_upper)
        alpha_unconstr = np.asarray(alpha_unconstr).ravel()

        alpha_1_unconstr = np.zeros(us_upper.shape).ravel()
        lambda_alpha_unconstr = np.zeros(us_upper.shape).ravel()

        if gamma_init_data:
            gamma_unconstr = np.clip(_softplus_inverse(us_upper / ms_upper), None, 10)
        else:
            gamma_unconstr = None

        self.module = VELOVAE(
            n_input=self.summary_stats["n_vars"],
            n_hidden=n_hidden,
            n_latent=n_latent,
            n_layers=n_layers,
            dropout_rate=dropout_rate,
            gamma_unconstr_init=gamma_unconstr,
            alpha_unconstr_init=alpha_unconstr,
            alpha_1_unconstr_init=alpha_1_unconstr,
            lambda_alpha_unconstr_init=lambda_alpha_unconstr,
            switch_spliced=ms_upper,
            switch_unspliced=us_upper,
            linear_decoder=linear_decoder,
            **model_kwargs,
        )
        self._model_summary_string = (
            "VELOVI Model with the following params: \nn_hidden: {}, n_latent: {}, n_layers: {}, dropout_rate: "
            "{}"
        ).format(
            n_hidden,
            n_latent,
            n_layers,
            dropout_rate,
        )
        self.init_params_ = self._get_init_params(locals())

    def train(
    self,
    max_epochs: Optional[int] = 10,
    use_gpu: Optional[bool] = None,
    train_size: float = 0.9,
    validation_size: Optional[float] = None,
    batch_size: int = 256,
    **kwargs,
    ):
        """
        Train the model.
        """
        # object to make train/test/val dataloaders
        data_splitter = DataSplitter(
            self.adata,
            train_size=train_size,
            validation_size=validation_size,
            batch_size=batch_size,
            use_gpu=use_gpu,
        )
        # defines optimizers, training step, val step, logged metrics
        training_plan = TrainingPlan(
            self.module, len(data_splitter.train_idx),
        )
        # creates Trainer, pre and post training procedures (Trainer.fit())
        runner = TrainRunner(
            self,
            training_plan=training_plan,
            data_splitter=data_splitter,
            max_epochs=max_epochs,
            use_gpu=use_gpu,
            **kwargs,
        )
        return runner()

    @classmethod
    @setup_anndata_dsp.dedent
    def setup_anndata(
        cls,
        adata: AnnData,
        spliced_layer: str,
        unspliced_layer: str,
        **kwargs,
    ) -> Optional[AnnData]:
        """
        %(summary)s.
        Parameters
        ----------
        %(param_adata)s
        spliced_layer
            Layer in adata with spliced normalized expression
        unspliced_layer
            Layer in adata with unspliced normalized expression

        Returns
        -------
        %(returns)s
        """
        setup_method_args = cls._get_setup_method_args(**locals())
        anndata_fields = [
            LayerField(REGISTRY_KEYS.X_KEY, spliced_layer, is_count_data=False),
            LayerField(REGISTRY_KEYS.U_KEY, unspliced_layer, is_count_data=False),
        ]
        adata_manager = AnnDataManager(
            fields=anndata_fields, setup_method_args=setup_method_args
        )
        adata_manager.register_fields(adata, **kwargs)
        cls.register_manager(adata_manager)

In [27]:
VELOVI.setup_anndata(adata, spliced_layer="Ms", unspliced_layer="Mu")

In [28]:
velo = VELOVI(adata)

	with soft mask.
Last Decoder layer: softmax


In [31]:
velo.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 1/500:   0%|          | 0/500 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (1043) must match the size of tensor b (0) at non-singleton dimension 1

Anndata setup with scvi-tools version 0.18.0.

Setup via `VELOVI.setup_anndata` with arguments:

{'spliced_layer': 'Ms', 'unspliced_layer': 'Mu'}

     Summary Statistics     
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Summary Stat Key ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│       n_U        │ 1043  │
│     n_cells      │ 3696  │
│      n_vars      │ 1043  │
└──────────────────┴───────┘

            Data Registry             
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃ scvi-tools Location ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│      U       │ adata.layers['Mu']  │
│      X       │ adata.layers['Ms']  │
└──────────────┴─────────────────────┘